<a href="https://colab.research.google.com/github/zakonreal/ds_homework/blob/main/NLP1_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Установка нужных пакетов
!pip install --upgrade nltk gensim bokeh umap-learn

import itertools
import string

import numpy as np
import umap
from nltk.tokenize import WordPunctTokenizer

from matplotlib import pyplot as plt

from IPython.display import clear_output

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 KB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=a66907c0ad7a97aeaf7c6eab954d9229cf26dc5c478f43fff60f8c185a676c1e
  Stored in directory: /root/.cache/pip/wheels/f4/3e/1c/596d0a463d17475af648688443fa4846fef624d1390339e7e9
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [2]:
# выгружаем датасет:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc

--2023-03-17 09:36:08--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.7.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.7.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2023-03-17 09:36:09--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc94fbfe99dc5f4b8be27e26632.dl.dropboxusercontent.com/cd/0/get/B4Z4PEqZj1Lr_k_DYwlb_hGdVRciA7MQVe_lW3BY_WsUE6PaTfBdYKgtqdcTmWBLDuyWXVnQlCwasZU-7sb9J882dIpzqL9nzKy85_iappvCIIJrZtVxqiSRwLn3KPcm80NmulFcN3d1Cn8D2rXBtNnLUDmDE9OuyygBl90BG903YA/file?dl=1# [following]
--2023-03-17 09:36:09--  https://ucc94fbfe99dc5f4b8be27e26632.dl.dropboxusercontent.com/cd/0/get/B4Z4PEqZj1Lr_k_DYwlb_hGdVRciA7MQVe_lW3BY_WsUE6PaTfBdYKgtqdcTmWBLDuyWXVnQlCwasZU-7sb9J8

In [3]:
data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

In [4]:
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


#Задание 1: Перевести все слова в нижний регистр (NLTK) из data и добавьте как лист токенов в листе data_tok


In [5]:
data_tok = list()
for i in data:
    token = tokenizer.tokenize(i)
    r = list()
    for e in token:
        r.append(e.lower())
    data_tok.append(r)

#checking

assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [6]:
data_tok1 = [[word.lower() for word in tokenizer.tokenize(line)] for line in data]

assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

###Задание 2: Подсчитайте топ10 самых популярных лем в рамках data

In [7]:
import collections
all_words = []
for i in data_tok:
  for j in i:
    all_words.append(j)
counter = collections.Counter(all_words)
counter.most_common(10)

[('?', 552413),
 ('the', 252068),
 ('what', 214798),
 ('is', 185392),
 ('a', 155726),
 ('i', 149735),
 ('to', 141788),
 ('in', 139787),
 ('how', 135687),
 ('of', 112001)]

In [8]:
list_data = []
for i in data_tok:
    list_data += i

dict_data = {}
for i in list_data:
    lemm = lemmatizer.lemmatize(i)
    if lemm not in dict_data:
        dict_data[lemm] = dict_data.get(lemm, 0)
    dict_data[lemm] += 1

top = []
for i in dict_data:
    top += [[i, dict_data[i]]]
sorted(top, key=lambda x: x[1], reverse = True)[:10]

[['?', 552413],
 ['the', 252068],
 ['what', 214798],
 ['is', 185392],
 ['a', 172513],
 ['i', 149868],
 ['to', 141788],
 ['in', 139813],
 ['how', 135687],
 ['of', 112001]]

###Задание 3: Подсчитайте количество разных слов до и после лемматизации

In [9]:
# all_words = []
# for i in data_tok:
#   for j in i:
#     all_words.append(j)

all_lemma = []
for i in all_words:
    t = lemmatizer.lemmatize(i)
    all_lemma.append(t)

unique_w = set(all_words)
len(unique_w), len(set(all_lemma))

(87820, 80304)

In [10]:
all_lemma = [lemmatizer.lemmatize(i) for i in all_words]
len(unique_w), len(set(all_lemma))

(87820, 80304)

In [25]:
# dict = {}
# for lem in all_lemma:
#     if lem in dict.keys():
#       dict[lem]+=1
#     else:
#       dict[lem]=1
# print(len(dict))
# print(dict)

# import pandas as pd

# df = pd.DataFrame(dict.items())
# df
# sorted_df = df.sort_values(by=1, ascending = False) 
# print(sorted_df[0:10])

###Задание 4: Подсчитайте количество разных слов до и после стемминга

In [12]:
all_stem = []
for i in all_words:
    t = ps.stem(i)
    all_stem.append(t)

unique_w = set(all_words)
len(unique_w), len(set(all_stem))

(87820, 67026)

In [13]:
all_stem = [ps.stem(i) for i in all_words]
len(unique_w), len(set(all_stem))

(87820, 67026)

REGEXP

https://www.programiz.com/python-programming/regex 

https://docs.python.org/3/howto/regex.html

In [15]:
import re

pattern = 'a[A-z]*s'
test_string = 'abyss'
result = re.match(pattern, test_string)

if result:
  print("Search successful.")
else:
  print("Search unsuccessful.")	

#debugging https://regex101.com/

Search successful.


###Задание 6: 
https://www.hackerrank.com/challenges/matching-specific-string/problem?isFullScreen=true 

###Задание 7: 
https://www.hackerrank.com/challenges/matching-whitespace-non-whitespace-character/problem?isFullScreen=true

###Задание 8: 
https://www.hackerrank.com/challenges/matching-start-end/problem?isFullScreen=true

###Задание 9: 
https://www.hackerrank.com/challenges/matching-word-boundaries/problem?isFullScreen=true

In [16]:
# Задание 6
# r"hackerrank"

In [17]:
# Задание 7
# r"\S\S\s\S\S\s\S\S"

In [18]:
# Задание 8
# r"^\d\w{4}\.$"

In [19]:
# Задание 9
# r'\b[aeiouAEIOU][a-zA-Z]*\b'

Bag Of Words (BOW)

In [ ]:
# def vectorize(tokens):
#     ''' This function takes list of words in a sentence as input 
#     and returns a vector of size of filtered_vocab.It puts 0 if the 
#     word is not present in tokens and count of token if present.'''
#     vector=[]
#     for w in filtered_vocab:
#         vector.append(tokens.count(w))
#     return vector
# def unique(sequence):
#     '''This functions returns a list in which the order remains 
#     same and no item repeats.Using the set() function does not 
#     preserve the original ordering,so i didnt use that instead'''
#     seen = set()
#     return [x for x in sequence if not (x in seen or seen.add(x))]

# #create a list of stopwords.You can import stopwords from nltk too
# stopwords=["to","is","a"]

# #list of special characters.You can use regular expressions too
# special_char=[",",":"," ",";",".","?"]

# #Write the sentences in the corpus,in our case, just two 
# string1="Welcome to Great Learning , Now start learning"
# string2="Learning is a good practice"

# #convert them to lower case
# string1=string1.lower()
# string2=string2.lower()

# #split the sentences into tokens
# tokens1=string1.split()
# tokens2=string2.split()
# print(tokens1)
# print(tokens2)

# #create a vocabulary list
# vocab=unique(tokens1+tokens2)
# print(vocab)

# #filter the vocabulary list
# filtered_vocab=[]
# for w in vocab: 
#     if w not in stopwords and w not in special_char: 
#         filtered_vocab.append(w)
# print(filtered_vocab)

# #convert sentences into vectords
# vector1=vectorize(tokens1)
# print(vector1)
# vector2=vectorize(tokens2)
# print(vector2)

Задание 10: Реализовать Bag of words на data_tok (можно на NLTK, можно без)

###Домашнее задание NLP1

###Задание 5: Подсчитайте количество разных слов
нужно посчитать (на базе предыдущих упражнений)
- до лемматизации и стеминнга
- после стемминга
- после лемматизации
- после стеминга и лемматизации

и отдельно сравнить (для важности корректного порядка процедур)

1. сначала стеминг, потом лемматизацию
2. сначала лемматизацию, потом стемминг

In [14]:
lemma_stem = []
for i in all_words:
    lemma_stem.append(ps.stem(lemmatizer.lemmatize(i)))

stem_lemma = []
for i in all_words:
    stem_lemma.append(lemmatizer.lemmatize(ps.stem(i)))

len(set(lemma_stem)), len(set(stem_lemma))

(66835, 66818)

In [20]:
# ML1_1:https://www.hackerrank.com/challenges/capturing-non-capturing-groups/problem?isFullScreen=true

# r'okokok'

In [21]:
# ML1_2: https://www.hackerrank.com/challenges/branch-reset-groups/problem?isFullScreen=true

# ?php

# $Regex_Pattern = '/^\d{2}(?|(-)|(:)|(---)|(.))\d{2}\1\d{2}\1\d{2}$/'; //Do not delete '/'. Replace __________ with your regex.

# $handle = fopen ("php://stdin","r");

In [23]:
# ML1_3: https://www.hackerrank.com/challenges/detect-html-links/problem?isFullScreen=true

# import re
# Regex_Pattern = r'(?<=<a href=")([^"]+)".*?>([^<]*)(?=</)'

# for i in range(int(input())):
#     matches = re.findall(Regex_Pattern, str(input()))
#     if matches:
#         for match in matches:
#             print(match[0],match[1].strip(),sep=',')

In [24]:
# ML1_4: Реализовать stemming, lemmatization & BoW на следующем датасете:
#  https://cloud.mail.ru/public/Z4L3/vB8GcgTtK (Russian Toxic-abuse comments)